In [15]:
pip install scikit-learn rank_bm25 pandas


In [20]:
# 导入必要的库
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# 1. 加载数据
file_path = 'doubanbook_top250_comments.txt'
data = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')

# 查看数据结构
data.head()

,book,id,star,time,likenum,body
0,天才在左 疯子在右,auntbear,allstar10,2013-07-29,1455.0,带着了解精神病群体的期待看这本书，前几篇还像回事儿，越往后就变成鬼故事和科幻小说了，严重的被...
1,天才在左 疯子在右,warfalcon,allstar10,2014-10-09,886.0,这是本起点小说，而不是纪实文学，如果你是一个科幻小说爱好者，会发现太多熟悉的情节和故事，过于...
2,天才在左 疯子在右,smallbad,allstar10,2013-11-27,723.0,给这本网路小说打四星以上的文化水平应该不高，没看过什么硬科幻，也不懂精神分析。真以为这是“访...
3,天才在左 疯子在右,ankazen,allstar20,2012-06-29,674.0,这本书证明半吊子这种人最危险。
4,天才在左 疯子在右,lecia,allstar10,2012-12-21,611.0,瞎编乱造，其他领域不敢说，但我敢说高铭就心理学上连个系统的了解都欠缺，编造的色彩很浓厚，还要...


In [24]:
# 2. 数据预处理
# 清洗评论文本
def clean_text(text):
    if not isinstance(text, str):  # 检查是否为字符串
        return ''  # 如果不是字符串，返回空字符串
    text = re.sub(r'\s+', ' ', text)  # 去除多余的空格
    text = re.sub(r'[^\w\s]', '', text)  # 移除标点符号
    text = text.lower()  # 转小写
    return text
# 对评论列进行清洗
data['cleaned_body'] = data['body'].apply(clean_text)

data['cleaned_body']

,cleaned_body
0,带着了解精神病群体的期待看这本书前几篇还像回事儿越往后就变成鬼故事和科幻小说了严重的被愚弄感...
1,这是本起点小说而不是纪实文学如果你是一个科幻小说爱好者会发现太多熟悉的情节和故事过于神话精神...
2,给这本网路小说打四星以上的文化水平应该不高没看过什么硬科幻也不懂精神分析真以为这是访谈录的大...
3,这本书证明半吊子这种人最危险
4,瞎编乱造其他领域不敢说但我敢说高铭就心理学上连个系统的了解都欠缺编造的色彩很浓厚还要打着是实...
...,...
34778,真的是强大的一本书
34779,作为处女作来说雕琢痕迹太重
34780,不喜欢
34781,展现了阿富汗生活历史画卷也表达了对自我的救赎故事也曲折老爸死了才知道自己的仆人是同父异母的弟...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import jieba
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class BookRecommender:
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')
        print("原始数据样例：")
        print(self.data.head())
        self.books = self.data['book'].unique()
        self._preprocess_data()
        self._build_models()

    def _preprocess_data(self):
        """数据预处理（增加分词过程输出）"""
        def clean_text(text):
            if not isinstance(text, str): return ''
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'[^\w\s]', '', text)
            return text.lower()

        def chinese_tokenizer(text):
            tokens = jieba.cut(text)
            return ' '.join(tokens)

        self.data['cleaned_body'] = self.data['body'].apply(clean_text)
        self.data['tokenized_body'] = self.data['cleaned_body'].apply(chinese_tokenizer)

        # 展示预处理前后的对比
        print("\n预处理前后对比样例：")
        print(self.data[['body', 'cleaned_body', 'tokenized_body']].head())

        self.book_comments = self.data.groupby('book')['tokenized_body'].apply(lambda x: ' '.join(x)).reset_index()
        print("\n按书籍聚合后的评论：")
        print(self.book_comments.head())

    def _build_models(self):
        """模型构建（增加特征输出）"""
        # TF-IDF模型
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.book_comments['tokenized_body'])

        # 输出TF-IDF特征词
        print("\nTF-IDF特征词示例：")
        print(self.tfidf_vectorizer.get_feature_names_out()[:20])

        # BM25模型
        tokenized_corpus = [doc.split() for doc in self.book_comments['tokenized_body']]
        self.bm25 = BM25Okapi(tokenized_corpus)

        # 输出BM25参数
        print("\nBM25模型参数：")
        print(f"文档平均长度: {self.bm25.avgdl:.2f}")
        print(f"文档数量: {len(self.bm25.doc_len)}")

    def _show_tfidf_calculation(self, query, top_n=3):
        """展示TF-IDF计算过程"""
        print("\n" + "="*50)
        print("TF-IDF 计算过程：")

        # 1. 查询预处理
        cleaned_query = re.sub(r'[^\w\s]', '', query.lower())
        tokenized_query = ' '.join(jieba.cut(cleaned_query))
        print(f"\n1. 查询预处理结果: '{tokenized_query}'")

        # 2. 向量化
        query_vec = self.tfidf_vectorizer.transform([tokenized_query])
        print("\n2. 查询向量化结果:")
        print(pd.DataFrame(query_vec.toarray(),
                         columns=self.tfidf_vectorizer.get_feature_names_out()).T.sort_values(0, ascending=False).head(10))

        # 3. 计算相似度
        similarities = cosine_similarity(query_vec, self.tfidf_matrix)
        print("\n3. 相似度计算结果:")
        print(pd.DataFrame({
            'book': self.book_comments['book'],
            'similarity': similarities[0]
        }).sort_values('similarity', ascending=False).head(top_n))

        return np.argsort(similarities[0])[-top_n:][::-1]

    def _show_bm25_calculation(self, query, top_n=3):
        """展示BM25计算过程"""
        print("\n" + "="*50)
        print("BM25 计算过程：")

        # 1. 查询预处理
        cleaned_query = re.sub(r'[^\w\s]', '', query.lower())
        query_tokens = list(jieba.cut(cleaned_query))
        print(f"\n1. 查询分词结果: {query_tokens}")

        # 2. 计算评分
        scores = self.bm25.get_scores(query_tokens)
        print("\n2. 评分计算结果:")
        print(pd.DataFrame({
            'book': self.book_comments['book'],
            'score': scores
        }).sort_values('score', ascending=False).head(top_n))

        return np.argsort(scores)[-top_n:][::-1]

    def compare_recommendations(self, query, top_n=3):
        """完整推荐过程"""
        print("\n" + "="*50)
        print(f"【查询】: '{query}'")

        # TF-IDF推荐
        print("\n>>> TF-IDF 推荐 <<<")
        tfidf_indices = self._show_tfidf_calculation(query, top_n)
        tfidf_rec = self.book_comments.iloc[tfidf_indices]['book'].tolist()

        # BM25推荐
        print("\n>>> BM25 推荐 <<<")
        bm25_indices = self._show_bm25_calculation(query, top_n)
        bm25_rec = self.book_comments.iloc[bm25_indices]['book'].tolist()

        # 对比结果
        print("\n" + "-"*20 + " 最终推荐结果 " + "-"*20)
        print(f"{'排名':<5}{'TF-IDF':<30}{'BM25':<30}")
        for i in range(top_n):
            print(f"{i+1:<5}{tfidf_rec[i][:25]:<30}{bm25_rec[i][:25]:<30}")

# 使用示例
if __name__ == "__main__":
    recommender = BookRecommender('doubanbook_top250_comments.txt')

    test_queries = ["科幻", "爱情", "历史"]
    for query in test_queries:
        recommender.compare_recommendations(query)
        input("\n按Enter键继续查看下一个查询结果...")

原始数据样例：
        book         id       star        time  likenum  \
0  天才在左 疯子在右   auntbear  allstar10  2013-07-29   1455.0   
1  天才在左 疯子在右  warfalcon  allstar10  2014-10-09    886.0   
2  天才在左 疯子在右   smallbad  allstar10  2013-11-27    723.0   
3  天才在左 疯子在右    ankazen  allstar20  2012-06-29    674.0   
4  天才在左 疯子在右      lecia  allstar10  2012-12-21    611.0   

                                                body  
0  带着了解精神病群体的期待看这本书，前几篇还像回事儿，越往后就变成鬼故事和科幻小说了，严重的被...  
1  这是本起点小说，而不是纪实文学，如果你是一个科幻小说爱好者，会发现太多熟悉的情节和故事，过于...  
2  给这本网路小说打四星以上的文化水平应该不高，没看过什么硬科幻，也不懂精神分析。真以为这是“访...  
3                                    这本书证明半吊子这种人最危险。  
4  瞎编乱造，其他领域不敢说，但我敢说高铭就心理学上连个系统的了解都欠缺，编造的色彩很浓厚，还要...  

预处理前后对比样例：
                                                body  \
0  带着了解精神病群体的期待看这本书，前几篇还像回事儿，越往后就变成鬼故事和科幻小说了，严重的被...   
1  这是本起点小说，而不是纪实文学，如果你是一个科幻小说爱好者，会发现太多熟悉的情节和故事，过于...   
2  给这本网路小说打四星以上的文化水平应该不高，没看过什么硬科幻，也不懂精神分析。真以为这是“访...   
3                                    这本书证明半吊子这种人最危险。   
4  瞎编乱造，其他领域不敢说